# **Ingestion del physical_specs.json**

##### **01.- Leer archivo JSON usando dataframereader de spark**

In [2]:
#p_file_date = "2025-07-07"
#p_environment = "Development"

StatementMeta(, 8a9d7b18-6a30-406d-8269-c3972b505599, 4, Finished, Available, Finished)

In [3]:
%run configuration

StatementMeta(, 8a9d7b18-6a30-406d-8269-c3972b505599, 5, Finished, Available, Finished)

In [4]:
%run common_functions

StatementMeta(, 8a9d7b18-6a30-406d-8269-c3972b505599, 6, Finished, Available, Finished)

In [5]:
#Importaciones de los tipos de datos para definir el esquema
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType

StatementMeta(, 8a9d7b18-6a30-406d-8269-c3972b505599, 7, Finished, Available, Finished)

In [6]:
# Esta es la estructura del df, esto se define para que inferSchema no haga un proceso extra 
# sirve para grandes volumenes de datos, asi se demoran menos en cargar
physical_schema = StructType ( fields = [
    StructField("PhysicalSpecID", IntegerType(), False),
    StructField("Width", StringType(), True),
    StructField("Height", StringType(), True),
    StructField("Depth", StringType(), True),
    StructField("BoundingVolume", StringType(), True),
    StructField("Mass", StringType(), True),
    StructField("RAMCapacity", StringType(), True),
    StructField("NonVolatileMemoryCapacity", StringType(), True),
    StructField("CellularController", StringType(), True),
    StructField("NominalBatteryCapacity", StringType(), True),
    StructField("EstimatedBatteryLife", StringType(), True)
])

StatementMeta(, 8a9d7b18-6a30-406d-8269-c3972b505599, 8, Finished, Available, Finished)

In [7]:
physical_path = f"{bronze_folder_path}/{p_file_date}/physical_specs"

df = (
    spark.read
        .schema(physical_schema)     
        .option("multiline", "true")
        .json(physical_path)          
)

StatementMeta(, 8a9d7b18-6a30-406d-8269-c3972b505599, 9, Finished, Available, Finished)

##### **2.- Eliminar columnas que no sirven**

In [8]:
df2 = df.drop("BoundingVolume", "NonVolatileMemoryCapacity", "EstimatedBatteryLife")

StatementMeta(, 8a9d7b18-6a30-406d-8269-c3972b505599, 10, Finished, Available, Finished)

##### **3.- Renombrar columnas**

In [9]:
new_cols = ["physical_spec_id", "width", "height", "depth", "mass", "ram_capacity", "cellular_controller", "battery_capacity"]

df3 = df2.toDF(*new_cols)

StatementMeta(, 8a9d7b18-6a30-406d-8269-c3972b505599, 11, Finished, Available, Finished)

##### **4.- Incluir ingestion_date y environment**

In [10]:
from pyspark.sql.functions import lit

df4 = add_ingestion_date(df3) \
        .withColumn("environment", lit(p_environment)) \
        .withColumn("file_date", lit(p_file_date))

StatementMeta(, 8a9d7b18-6a30-406d-8269-c3972b505599, 12, Finished, Available, Finished)

In [11]:
%%sql
DROP TABLE if EXISTS physical_specs

StatementMeta(, 8a9d7b18-6a30-406d-8269-c3972b505599, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

##### **5.- Pasar a delta y enviar a lh_silver**

In [12]:
df4.write.format("delta").mode("overwrite").saveAsTable("physical_specs")

StatementMeta(, 8a9d7b18-6a30-406d-8269-c3972b505599, 14, Finished, Available, Finished)

In [13]:
%%sql
SELECT * from physical_specs
limit(10)

StatementMeta(, 8a9d7b18-6a30-406d-8269-c3972b505599, 15, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 11 fields>